#  **<center>TP : Analyse Exploratoire Interactive avec Plotly<center>**

---

##  **Dataset : Superstore Sales**

**Source** : [Kaggle - Sample Superstore Dataset](https://www.kaggle.com/datasets/vivek468/superstore-dataset-final)

**Description** : Données de ventes d'un magasin retail américain avec :
- **Dimensions temporelles** : Order Date, Ship Date
- **Dimensions géographiques** : Country, State, City, Region
- **Dimensions catégorielles** : Category, Sub-Category, Segment
- **Métriques** : Sales, Profit, Quantity, Discount

**Colonnes principales** :
```
- Order ID, Order Date, Ship Date
- Customer ID, Customer Name, Segment
- Country, City, State, Region
- Product ID, Category, Sub-Category, Product Name
- Sales, Quantity, Discount, Profit

---

## **Imports et chargement du dataset Superstore**

In [ ]:
# Importer les bibliothèques nécessaires
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from pathlib import Path
import kagglehub
import os

C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Télécharger la dernière version du dataset Superstore depuis Kaggle
path = kagglehub.dataset_download("vivek468/superstore-dataset-final")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\ACER\.cache\kagglehub\datasets\vivek468\superstore-dataset-final\versions\1


In [ ]:
# Lister les fichiers du dataset
files = os.listdir(path)
print("\nFichiers disponibles:")
for file in files:
    print(f"{file}")


Fichiers disponibles:
Sample - Superstore.csv


In [ ]:
# Charger le dataset principal (fichier CSV)
csv_files = [f for f in files if f.endswith('Sample - Superstore.csv')]
print(f"\nFichiers CSV trouvés: {csv_files}")


Fichiers CSV trouvés: ['Sample - Superstore.csv']


In [ ]:
# Charger les données
df = pd.read_csv(os.path.join(path, csv_files[0]), encoding='latin-1')
print(f"Dataset chargé: {csv_files[0]} | {df.shape[0]:,} lignes, {df.shape[1]} colonnes")

Dataset chargé: Sample - Superstore.csv | 9,994 lignes, 21 colonnes


In [ ]:
# Nettoyage minimal
# Harmoniser les noms de colonnes (minuscules, underscore)
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
)

In [ ]:
# Colonnes numériques à forcer
numeric_cols = ["sales", "profit", "quantity", "discount"]
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

In [ ]:
# Afficher les premières lignes du dataset
df.head()

,row_id,order_id,order_date,ship_date,ship_mode,customer_id,customer_name,segment,country,city,...,postal_code,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


---

## **1. Exercice 1 — Prise en main**

In [ ]:
# 1.1 Aperçu et informations générales
preview_cols = ["order_id", "order_date", "ship_date", "category", "sub-category", "segment", "sales", "profit", "quantity", "discount"]
print("Premières lignes:\n", df[preview_cols].head())
print("\nInfo dataset:")
print(df.info())

Premières lignes:
          order_id  order_date   ship_date         category sub-category  \
0  CA-2016-152156   11/8/2016  11/11/2016        Furniture    Bookcases   
1  CA-2016-152156   11/8/2016  11/11/2016        Furniture       Chairs   
2  CA-2016-138688   6/12/2016   6/16/2016  Office Supplies       Labels   
3  US-2015-108966  10/11/2015  10/18/2015        Furniture       Tables   
4  US-2015-108966  10/11/2015  10/18/2015  Office Supplies      Storage   

     segment     sales    profit  quantity  discount  
0   Consumer  261.9600   41.9136         2      0.00  
1   Consumer  731.9400  219.5820         3      0.00  
2  Corporate   14.6200    6.8714         2      0.00  
3   Consumer  957.5775 -383.0310         5      0.45  
4   Consumer   22.3680    2.5164         2      0.20  

Info dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------

In [ ]:
# 1.2 Afficher les valeurs manquantes par colonne
print("\nValeurs manquantes :")
missing_values = df.isnull().sum()
for col, count in missing_values.items():
    if count > 0:
        print(f"  - {col}: {count} valeurs manquantes")
    else:
        print(f"  - {col}: 0 valeurs manquantes")


Valeurs manquantes :
  - row_id: 0 valeurs manquantes
  - order_id: 0 valeurs manquantes
  - order_date: 0 valeurs manquantes
  - ship_date: 0 valeurs manquantes
  - ship_mode: 0 valeurs manquantes
  - customer_id: 0 valeurs manquantes
  - customer_name: 0 valeurs manquantes
  - segment: 0 valeurs manquantes
  - country: 0 valeurs manquantes
  - city: 0 valeurs manquantes
  - state: 0 valeurs manquantes
  - postal_code: 0 valeurs manquantes
  - region: 0 valeurs manquantes
  - product_id: 0 valeurs manquantes
  - category: 0 valeurs manquantes
  - sub-category: 0 valeurs manquantes
  - product_name: 0 valeurs manquantes
  - sales: 0 valeurs manquantes
  - quantity: 0 valeurs manquantes
  - discount: 0 valeurs manquantes
  - profit: 0 valeurs manquantes


In [ ]:
# 1.3 Conversion des dates
for date_col in ["order_date", "ship_date"]:
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce", infer_datetime_format=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_29024\645532758.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_col] = pd.to_datetime(df[date_col], errors="coerce", infer_datetime_format=True)
C:\Users\ACER\AppData\Local\Temp\ipykernel_29024\645532758.py:4: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df[date_col] = pd.to_datetime(df[date_col], errors="coerce", infer_datetime_format=True)


In [ ]:
# 1.4 Création de colonnes utiles
if "order_date" in df.columns:
    df["year_month"] = df["order_date"].dt.to_period("M").astype(str)
if "sales" in df.columns:
    df["revenue"] = df["sales"]

In [ ]:
# 1.5 Statistiques descriptives
print("\nStatistiques principales (numeriques):\n", df[numeric_cols].describe())


Statistiques principales (numeriques):
               sales       profit     quantity     discount
count   9994.000000  9994.000000  9994.000000  9994.000000
mean     229.858001    28.656896     3.789574     0.156203
std      623.245101   234.260108     2.225110     0.206452
min        0.444000 -6599.978000     1.000000     0.000000
25%       17.280000     1.728750     2.000000     0.000000
50%       54.490000     8.666500     3.000000     0.200000
75%      209.940000    29.364000     5.000000     0.200000
max    22638.480000  8399.976000    14.000000     0.800000


### Visualisation 1 : Évolution temporelle des ventes

In [ ]:
# 1.6 Visualisation 1 : Évolution temporelle des ventes mensuelles
if "year_month" not in df.columns:
    raise ValueError("La colonne year_month est requise. Vérifiez la conversion des dates.")

monthly_sales = df.groupby("year_month").agg({
    "revenue": "sum",
    "order_id": "nunique"
}).reset_index().sort_values("year_month")

fig_line = px.line(
    monthly_sales,
    x="year_month",
    y="revenue",
    markers=True,
    title="Évolution mensuelle des ventes",
    labels={"year_month": "Mois", "revenue": "Ventes totales"},
    hover_data={"order_id": True, "revenue": ":.2f"}
)
fig_line.update_layout(xaxis_tickangle=-45)
fig_line.show()

### Observations :
1. **Croissance continue** : Augmentation de 15k (2014) à 120k (pic 2017), soit une multiplication par 8.

2. **Forte volatilité** : Fluctuations mensuelles importantes (±50k), indiquant une demande instable.

3. **Saisonnalité marquée** : 
   - Pics systématiques en Q4 (fêtes de fin d'année)
   - Creux en Q1 (janvier-février)

4. **Performance exceptionnelle fin 2017** : Pic à 120k, dépassant les précédents records de 50%.

### Visualisation 2 : Distribution par catégorie

In [ ]:
# 1.6 Visualisation 2 : Ventes par catégorie (barres horizontales triées)
cat_sales = df.groupby("category", as_index=False)["revenue"].sum().sort_values("revenue", ascending=False)
fig_cat = px.bar(
    cat_sales,
    x="revenue",
    y="category",
    orientation="h",
    text="revenue",
    title="Ventes par catégorie",
    labels={"revenue": "Ventes", "category": "Catégorie"},
)
fig_cat.update_traces(texttemplate="%{text:.2s}", textposition="outside")
fig_cat.update_layout(yaxis=dict(categoryorder="total ascending"))
fig_cat.show()

### Observations :

#### Distribution équilibrée
1. **Faible écart entre catégories** : Les 3 catégories génèrent des ventes similaires (720k-840k), avec seulement 120k d'écart (14% de différence).

2. **Technology en tête** : 840k de ventes, soit le segment le plus performant (+13% vs Furniture, +17% vs Office Supplies).

3. **Furniture et Office Supplies proches** : Différence minime de 20k entre ces deux catégories (~3%), indiquant un marché équilibré.

#### Implications stratégiques
- **Portefeuille diversifié** : Pas de dépendance excessive à une catégorie, réduisant le risque commercial.
- **Potentiel de Technology** : Catégorie leader, possibilité d'investir davantage pour creuser l'écart.
- **Opportunités d'optimisation** : Office Supplies et Furniture pourraient bénéficier d'actions ciblées pour rattraper Technology.

### Visualisation 3 : Relation prix/quantité

In [ ]:
# 1.8 Visualisation 3 : Relation Sales vs Quantity avec tendance
fig_scatter = px.scatter(
    df,
    x="quantity",
    y="revenue",
    color="category",
    hover_data=["sub-category", "segment", "discount", "profit"],
    title="Relation Quantité vs Ventes",
    labels={"quantity": "Quantité", "revenue": "Ventes"},
    trendline="ols",
    opacity=0.7,
)
fig_scatter.show()

### Observations :

#### Corrélation et tendance
1. **Corrélation positive modérée** : Les lignes de tendance (OLS) montrent une relation croissante quantité/ventes pour les 3 catégories, mais avec forte dispersion.

2. **Outliers significatifs** : 
   - Technology : vente exceptionnelle à ~22k avec seulement 6 unités
   - Quelques transactions atteignent 10k-18k pour des quantités faibles (2-5 unités)

#### Comportement par catégorie
3. **Technology = produits premium** : Génère les ventes les plus élevées même à faible quantité, indiquant des prix unitaires supérieurs.

4. **Office Supplies & Furniture** : Ventes généralement <5k, suggérant des produits à moindre valeur unitaire.

#### Concentration des transactions
5. **Volume faible dominant** : La majorité des commandes portent sur 2-6 unités, rarement au-delà de 10.

6. **Dispersion verticale importante** : À quantité égale, les ventes varient énormément (facteur 10-100x), reflétant une grande hétérogénéité de prix dans le catalogue.

#### Implication stratégique
La valeur des ventes dépend davantage du **type de produit** que de la **quantité commandée**, particulièrement pour Technology.

---

## **2. Exercice 2 — Interactivité avancée**

In [ ]:
# 2.1 Préparation des données pour l'animation
if "profit" not in df.columns:
    raise ValueError("La colonne 'profit' est requise pour l'exercice 2.")

agg_month_cat = (
    df.groupby(["year_month", "category"], as_index=False)
    .agg({"revenue": "sum", "profit": "sum", "quantity": "sum"})
    .sort_values("year_month")
)

### Visualisation 4 : Scatter plot animé

In [ ]:
# 2.2 Scatter plot animé (Sales vs Profit, taille ~ Quantity)
fig_anim = px.scatter(
    agg_month_cat,
    x="revenue",
    y="profit",
    animation_frame="year_month",
    animation_group="category",
    size="quantity",
    color="category",
    hover_name="category",
    hover_data={"revenue": ":.2f", "profit": ":.2f", "quantity": True, "year_month": True},
    labels={"revenue": "Ventes", "profit": "Profit"},
    title="Évolution mensuelle Sales vs Profit par catégorie (animé)",
    height=600,
)
fig_anim.show()

### Observations :

#### Relation Sales-Profit
1. **Corrélation positive forte** : Les points se situent globalement dans la zone haute-droite, indiquant qu'une augmentation des ventes accompagne une augmentation du profit.

2. **Profitabilité hétérogène** : À ventes égales, le ratio profit/ventes varie selon la catégorie, révélant des structures de coûts ou des marges différentes.

#### Comportement par catégorie
3. **Technology dominant** : Génère les plus hautes ventes et profits mensuels, positionnée en haut-droite du graphique.

4. **Office Supplies & Furniture** : Regroupés en bas-gauche avec ventes/profits plus faibles et plus stables dans le temps.

#### Implications stratégiques
- **Technology est le moteur** : Prioriser l'optimisation et l'expansion de cette catégorie
- **Marge vs Volume** : Technology a une meilleure profitabilité relative

### Visualisation 5 : Dropdown interactif

In [ ]:
# ============================================================================
# 2.3 DROPDOWN MENU (Menu déroulant) - Sales / Profit / Quantity
# ============================================================================

print("\n📊 Dropdown pour changer de métrique")

# Agrégation par sous-catégorie pour plus de granularité
agg_subcat = df.groupby("sub-category", as_index=False)[["revenue", "profit", "quantity"]].sum()
agg_subcat = agg_subcat.sort_values("revenue", ascending=False)

# Créer plusieurs traces
fig_dropdown = go.Figure()

# Trace 1 : Sales (Ventes)
fig_dropdown.add_trace(
    go.Bar(
        x=agg_subcat["revenue"],
        y=agg_subcat["sub-category"],
        orientation="h",
        name="Sales (Ventes)",
        visible=True,  # Visible par défaut
        text=agg_subcat["revenue"],
        textposition="outside",
    )
)

# Trace 2 : Profit (Bénéfices)
fig_dropdown.add_trace(
    go.Bar(
        x=agg_subcat["profit"],
        y=agg_subcat["sub-category"],
        orientation="h",
        name="Profit (Bénéfices)",
        visible=False,  # Caché par défaut
        text=agg_subcat["profit"],
        textposition="outside",
    )
)

# Trace 3 : Quantity (Quantités)
fig_dropdown.add_trace(
    go.Bar(
        x=agg_subcat["quantity"],
        y=agg_subcat["sub-category"],
        orientation="h",
        name="Quantity (Quantités)",
        visible=False,  # Caché par défaut
        text=agg_subcat["quantity"],
        textposition="outside",
    )
)

# Ajouter le dropdown
fig_dropdown.update_layout(
    title="📊 Top sous-catégories - Sélectionnez la métrique",
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    label="Sales (Ventes)",
                    method="update",
                    args=[{"visible": [True, False, False]},
                          {"xaxis": {"title": "Sales (Ventes)"}}]
                ),
                dict(
                    label="Profit (Bénéfices)",
                    method="update",
                    args=[{"visible": [False, True, False]},
                          {"xaxis": {"title": "Profit (Bénéfices)"}}]
                ),
                dict(
                    label="Quantity (Quantités)",
                    method="update",
                    args=[{"visible": [False, False, True]},
                          {"xaxis": {"title": "Quantity (Quantités)"}}]
                )
            ]),
            direction="down",
            showactive=True,
            x=0.1,
            y=1.15
        )
    ],
    height=650,
    yaxis=dict(categoryorder="total ascending"),
)

# Export vers HTML pour visualisation garantie
fig_dropdown.write_html("dropdown_interactive.html")
print("✅ Graphique exporté vers 'dropdown_interactive.html'")

# Afficher dans le notebook (peut ne pas fonctionner dans certains environnements)
fig_dropdown.show()


📊 Dropdown pour changer de métrique
✅ Graphique exporté vers 'dropdown_interactive.html'


---

### **3. Exercice 3 — Dashboard multi-vues**

In [ ]:
# 3.1 Préparations des agrégations
# Série temporelle ventes/profit
monthly_perf = df.groupby("year_month", as_index=False).agg({"revenue": "sum", "profit": "sum"}).sort_values("year_month")

# Sub-category top 10
subcat_top10 = (
    df.groupby("sub-category", as_index=False)["revenue"].sum()
    .sort_values("revenue", ascending=False)
    .head(10)
)

# Matrice de corrélation
corr_cols = [col for col in ["revenue", "profit", "quantity", "discount"] if col in df.columns]
corr_matrix = df[corr_cols].corr()

# Box plot profit par catégorie
# 3.2 Dashboard 2x2
fig_dash = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Évolution mensuelle Ventes & Profit",
        "Top 10 Sous-catégories",
        "Corrélations (Sales/Profit/Quantity/Discount)",
        "Distribution du Profit par Catégorie",
    ),
    specs=[[{"type": "scatter"}, {"type": "bar"}],
           [{"type": "heatmap"}, {"type": "box"}]],
    horizontal_spacing=0.12,
    vertical_spacing=0.12,
)

# (1) Série temporelle
fig_dash.add_trace(
    go.Scatter(
        x=monthly_perf["year_month"],
        y=monthly_perf["revenue"],
        mode="lines+markers",
        name="Ventes",
        line=dict(color="#2c7be5"),
        fill="tozeroy",
    ),
    row=1, col=1,
)
fig_dash.add_trace(
    go.Scatter(
        x=monthly_perf["year_month"],
        y=monthly_perf["profit"],
        mode="lines+markers",
        name="Profit",
        line=dict(color="#28a745"),
        fill="tonexty",
    ),
    row=1, col=1,
)

# (2) Distribution sous-catégorie
fig_dash.add_trace(
    go.Bar(
        x=subcat_top10["revenue"],
        y=subcat_top10["sub-category"],
        orientation="h",
        name="Ventes",
        marker_color="#fd7e14",
    ),
    row=1, col=2,
)
fig_dash.update_yaxes(categoryorder="total ascending", row=1, col=2)

# (3) Heatmap corrélation
fig_dash.add_trace(
    go.Heatmap(
        z=corr_matrix.values,
        x=corr_matrix.columns,
        y=corr_matrix.columns,
        colorscale="RdBu",
        zmid=0,
        text=np.round(corr_matrix.values, 2),
        texttemplate="%{text}",
    ),
    row=2, col=1,
)

# (4) Box plot profit par catégorie
fig_dash.add_trace(
    go.Box(
        x=df["category"],
        y=df["profit"],
        boxpoints="outliers",
        marker_color="#9b59b6",
        name="Profit",
    ),
    row=2, col=2,
)

fig_dash.update_layout(
    height=900,
    showlegend=True,
    template="plotly_white",
    title_text="Dashboard interactif Superstore",
    margin=dict(t=80, l=60, r=40, b=60),
)
fig_dash.show()

### Observations :

#### Série temporelle (Top gauche)
- **Croissance nette** : Evolution de ~10k à 120k entre 2014-2017, avec pic fin 2017
- **Saisonnalité marquée** : Pics réguliers en Q4 (fêtes), creux en Q1
- **Profit vs Ventes** : Le profit suit les ventes mais avec une proportion plus faible (dispersion visible)

#### Top 10 Sous-catégories (Top droite)
**Top 3 stars** :
- Phones (445k) - Leader incontesté
- Chairs (260k) - Solide
- Storage (220k) - Bon volume

**Préoccupations** : Tables figure dans le top mais est déficitaire

#### Matrice de corrélations (Bottom gauche)
**Insights clés** :
- **Revenue ↔ Profit** : 0.48 (corrélation modérée) → ventes ≠ profitabilité
- **Revenue ↔ Quantity** : 0.2 (faible) → volume n'explique pas les ventes
- **Discount ↔ Profit** : -0.22 (négatif) → les réductions tuent la marge
- **Quantity ↔ Profit** : 0.07 (quasi-nul) → quantité n'impacte presque pas le profit

#### Distribution Profit (Bottom droite)
**Problèmes identifiés** :
- **Furniture** : Profit très limité (~0), peu de variabilité
- **Office Supplies** : TRÈS déficitaire avec outliers négatifs (-4000 à -2000)
- **Technology** : Meilleure profitabilité, mais plusieurs outliers négatifs

**Conclusion** : Office Supplies est un gouffre financier malgré les ventes !

In [ ]:
# 3.3 Export HTML
dashboard_html = "dashboard_final.html"
fig_dash.write_html(dashboard_html)
print(f"Dashboard exporté dans {dashboard_html}")

Dashboard exporté dans dashboard_final.html
